# Qiskit Mini-Projects and Libraries

In [299]:
# Classical Tools
"""
python libraries:
    numpy ----------------> math, matrices
    matplotlib/pyplot ----> visualization
    pandas ---------------> data handling
    scipy.optimize -------> for QAOA/VQE

    qiskit add-ons:
    qiskit.algorithms ----> VQE, QAOA, Grover, ...
    qiskit_nature --------> chemistry
    qiskit_optimization --> TSP, MaxCut, ...
"""

# Mini-Projects
"""
1. Grover's Search: find a secret item in a list
2. QAOA Optimization: Solve MaxCut/simple scheduling problem
3. Quantum-Enhanced Machine Learning: using hybrid classification
4. Real Molecule Energy Estimation: using VQE
5. Game/Simulation: with quantum logic
"""

"\n1. Grover's Search: find a secret item in a list\n2. QAOA Optimization: Solve MaxCut/simple scheduling problem\n3. Quantum-Enhanced Machine Learning: using hybrid classification\n4. Real Molecule Energy Estimation: using VQE\n5. Game/Simulation: with quantum logic\n"

# 1. Grover's Search:

In [300]:
# Goal: Find a secret 2-bit password using Grover's algorithm
"""
- Create quantum oracle that marks secret bitstring --> Secret Password = '11'
- Apply Grover's amplification to boost probability
- Measure & retrieve password
- Plot histogram of measurements
"""

from qiskit import QuantumCircuit, transpile
from qiskit_aer import Aer #new way of doing "from qiskit import Aer"

#define password:
target = '11' #secret password

#construct the oracle:
def password_oracle(circuit, target_bits): #oracle --> flips phase of (the correct answer) target state
    if target_bits == '00': #case 1
        circuit.z([0, 1])
        circuit.cz(0, 1)
        circuit.z([0, 1])
    elif target_bits == '01': #case 2
        circuit.x(0)
        circuit.cz(0, 1)
        circuit.x(0)
    elif target_bits == '10': #case 3
        circuit.x(1)
        circuit.cz(0, 1)
        circuit.x(1)
    elif target_bits == '11': #case 4
        circuit.cz(0, 1)

#Grover diffusion:
def diffuser(circuit): #grover diffusion operator: amplifies probability of target states & reduces others
    circuit.h([0, 1])
    circuit.x([0, 1])
    circuit.h(1)
    circuit.cx(0, 1)
    circuit.h(1)
    circuit.x([0, 1])
    circuit.h([0, 1])

In [301]:
#build Grover circuit:
qc = QuantumCircuit(2, 2)
qc.h([0, 1])

#apply the oracle and diffuser:
password_oracle(qc, target)
diffuser(qc)

#measure output:
qc.measure([0, 1], [0, 1])

In [302]:
#simulate the circuit (this method uses Qiskit's built-in Aer simulator, as oposed to using cloud/IBM quantum hardware):
simulator = Aer.get_backend('aer_simulator') #uses AerSimulator as circuit backend
compiled = transpile(qc, simulator) #transpiles circuit
job = simulator.run(compiled, shots = 1024) #new way instead of 'execute(compiled, backend = simulator, shots = 1024)

result = job.result()
counts = result.get_counts()

#output most probable results:
most_likely = max(counts, key = counts.get)
print("Cracked Password:", most_likely)

Cracked Password: 11


# 2. Quantum Approximate Optimization Algorithm (QAOA) Optimization: 

In [303]:
# Goal: Solve MaxCut or a simple scheduling problem

"""
- Given a graph where each edge has a cost --> divide graph's nodes into 2 groups so
# of cut edges (diagonals) is maximized (combinatorial optimization --> QAOA tackles this)
- Create graph with NetworkX
- Use Qiskit's QAOA tool
- Use classical optimizer to tweak quantum circuit parameters
- Visualize results
"""

import networkx as nx #creates graphs
from qiskit_optimization.applications import MaxCut
from qiskit_optimization.converters import QuadraticProgramToQubo
from qiskit_optimization import QuadraticProgram
import matplotlib.pyplot as plt

#crate a 4-node graph:
graph = nx.Graph()
edges = [(0, 1), (0, 2), (1, 2), (2, 3)]
graph.add_edges_from(edges)

#visualize graph:
nx.draw(graph, with_labels = True, node_color = 'lightblue', edge_color = 'gray')
plt.title("MaxCut Graph")
plt.show()

#convert to MaxCut QUBO problem:
maxcut = MaxCut(graph)
qubo = maxcut.to_quadratic_program()

ModuleNotFoundError: No module named 'networkx'

In [ ]:
#solve using QAOA:
from qiskit.algorithms import QAOA
from qiskit_optimization.algorithms import MinimumEigenOptimizer
from qiskit_aer import Aer
from qiskit.utils import algorithm_globals, QuantumInstance
from qiskit.algorithms.optimizers import COBYLA

#set up quantum simulator:
backend = Aer.get_backend('aer_simulator_statevector')
quantum_instance = QuantumInstance(backend)

#set up QAOA:
qaoa = QAOA(optimizer = COBYLA(), reps = 1, quantum_instance = quantum_instance) #higher reps increases accuracy and runtime
optimizer = MinimumEigenOptimizer(qaoa)

#solve MaxCut problem:
result = optimizer.solve(qubo)

#output solution:
print("Best partition (bitstring):", result.x)
print("MaxCut value (edges cut):", result.fval)

In [ ]:
#visualize the cut:
colors = ['red' if bit == 0 else 'blue for bit in result.x'] #sliced loop
nx.draw(graph, with_labels = True, node_color = colors, edge_color = 'black')
plt.title("QAOA MaxCut Partition")
plt.show()

# 3. Quantum-Classical Hybrid Classifier (QSVM/ Quantum Neural Network)

In [ ]:
# Goal: Build a binary classifier that learns to distinguish between 2 classes using a 
# variational quantum circuit and classical training

"""
- Given a dataset with 2 classes (0 & 1), train a quantum machine learning (ML) model to
classify new points correctly
- Start with 2D dataset (points in a plane), use parameterized quantum circuit as ML model
- Use use QNN to teach the VQC model
- hybrid train using classical optimizers

"""

import numpy as np
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler

#create a binary classification dataset (2 features):
x, y = make_classification(n_samples = 100, n_features = 2, n_informative = 2,
                           n_redundant = 0, n_clusters_per_class = 1, class_sep = 2.0
                           random_state = 42)

#normalize features to [0, pi]
scaler = MinMaxScaler(feature_range = 0, np.pi)
x_scaled = scaler.fit_transform(x)

#split into train/test
x_train, x_test, y_train, y_test = train_test_split(x_scaled, y, test_size = 0.2, random_state = 42)

In [ ]:
#build the quantum model:
from qiskit_machine_learning.algorithms import VQC
from qiskit_machine_learning.kernels import QuantumKernel
from qiskit_machine_learning.circuit.library import RawFeatureVector
from qiskit_machine_learning import TwoLocal
from qiskit_machine_learning.datasets import ad_hoc_data

from qiskit.algorithms.optimizers import COBYLA
from qiskit.utils import QuantumInstance
from qiskit_aer import Aer

#define feature map (how to encode data into a quantum state)
feature_map = RawFeatureVector(2)

#define variational form (learnable part)
ansatz = TwoLocal(2, ['ry', 'rz'], 'cz', reps = 2)

#set up simulator
backend = Aer.get_backend('aer_simulator_statevector')
quantum_instance = QuantumInstance(backend)

#define VQC model:
vqc = VQC(feature_map = feature_map, ansatz = ansatz, optimizer = COBYLA(), 
          quantum_instance = quantum_instance)

In [ ]:
#train the model:
from qiskit_machine_learning.algorithms.classifiers import NeuralNetworkClassifier

#use vqc as a classifier:
from qiskit_machine_learning.neural_networks import EstimatorQNN
from qiskit.primitives import Estimator

#build a QNN (quantum neural network):
qnn = EstimatorQNN(circuit = ansatz, input_params = feature_map.parameters, #as opposed to logistic regression
                   weight_params = ansatz.parameters, estimator = Estimator())

#wrap it as a scikit-learn classifier:
classifier = NeuralNetworkClassifier(qnn, optimizer = COBYLA(maxiter = 100))

#train:
classifier.fit(x_train, y_train) #fits the data to the regression line

#test accuracy:
accuracy = classifier.score(x_test, y_test)
print(f"Test Accuracy: {accuracy: .2f}")

In [ ]:
#visualization:
import matplotlib.pyplot as plt

def plot_2d(x, y, title = "Data"):
    plt.figure(figsize = (5, 5))
    plt.scatter(x[:, 0], x[:, 1], c = y, cmap = 'coolwarm', s = 40)
    plt.xlabel("Feature 1")
    plt.ylabel("Feature 2")
    plt.title(title)
    plt.grid(True)
    plt.show()

plot_2d(x_test, classifier.predict(x_test), title = "Predicted Classes")

# 4. Variational Quantum Eigensolver (VQE) to Simulate a Molecule

In [ ]:
# Goal: Use VQE to calculate the ground-state energy of a H₂ molecule with
# quantum chemistry using quantum circuits

"""
- Use Qiskit Nature --> for quantum chemistry
- Use VQE + variational ansatz + classical optimizer
- Run hybrid quantum-classical workflows
- Compare results to known values
"""

#define the molecule:
from qiskit_nature.second_q.drivers import PySCFDriver
from qiskit_nature.units import DistanceUnit

#create a driver to define modecule (H₂ with 0.735 Å bond length):
driver = PySCFDriver(atom = 'H 0 0 0; H 0 0 0.735',
                     basis = 'sto3g', unit = DistanceUnit.ANGSTROM)

problem = driver.run()

In [ ]:
#map to qubits:
from qiskit_nature.second_q.mappers import JordanWignerMapper
from qiskit_nature.second_q.circuit.library import HartreeFock

#map to qubits using Jordan-Wigner:
mapper = JordanWignerMapper()
qubit_op = mapper.map(problem.hamiltonian)

In [ ]:
#set up VQE:
from qiskit.algorithms.minimum_eigensolvers import VQE
from qiskit.algorithms.optimizers import SLSQP
from qiskit.circuit.library import TwoLocal
from qiskit.primitives import Estimator
from qiskit import Aer

#choose ansatz (trial wavefunction):
ansatz = TwoLocal(rotation_blocks = 'ry', entanglement_blocks = 'cz', entanglement = 'full', reps = 1)

#use VQE with classical optimizer:
vqe_solver = VQE(ansatz = ansatz, optimizer = SLSQP(), estimator = Estimator())

#solve:
result = vqe_solver.compute_minimum_eigenvalue(qubit_op)

print(f"Estimated ground state energy: {result.eigenvalue.real: .6f} Hartree")

In [ ]:
#compare to exact energy:
from qiskit.algorithms.minimum_eigensolvers import NumPyMinimumEigensolver

#use classical exact solver for reference:
exact_solver = NumPyMinimumEigensolver()
exact_result = exact_solver.compute_minimum_eigenvalue(qubit_op)

print(f"Exact ground state energyL {exact_result.eigenvalue.real: .6f} Hartree")

# 5. Quantum Tic-Tac-Toe

In [311]:
#winning system:
import numpy as np

def win_detection(post):
    game_won = False
    a = 0
    b = 0
    c = 0
    d = 2

    while a < 7 and game_won == False:
        if post[a] == post[a + 1] == post[a + 2]:
            game_won = True
        else:
            a = a + 3
    while b < 3 and game_won == False:
        if post[b] == post[b + 3] == post[b + 6]:
            game_won = True
        else:
            b = b + 1
    if post[c] == post[c + 4] == post[c + 7]:
        game_won = True
    if post[d] == post[d + 2] == post[d + 4]:
        game_won = True
    
    if game_won == True:
        print("Game over")
        i = 0
        while i < 9:
            print(post[i], post[i + 1], post[i + 2])
            i = i + 3
        return game_won
    
    # 1 2 3
    # 4 5 6
    # 7 8 9

    # 1,2,3 4,5,6 7,8,9
    # 1,4,7 2,5,8 3,6,9
    # 1,5,8 3,5,7

In [379]:
#simulate the circuit
def simulation(qc):
    backend = Aer.get_backend('aer_simulator')
    job = backend.run(transpile(qc, backend), shots = 1)
    result = job.result()
    counts = result.get_counts()
    outcome = list(counts.keys())[0]  #like '11' or '00'
    return outcome

In [436]:
# Goal: Build a game of Tic-Tac-Toe where moves can be superpositions, and the board
# collapses when measured, using quantum game logic, entanglement, and measurement

"""
- Each move is a qubit in superposition between "X --> |1>" and "O --> |0>"
- Measuring a tile collapses it to 1 value
- Quantum strategies (entangled moves, Grover-based tile selection, quantum randomness in moves)
- Start with 3x3 board, each cell is a qubit, simulate quantum moves
- Add entangled gameplay (2 tiles share an entangled state --> move in 1 tile influences result in another)
    Example of this:
    qc.h(0)
    qc.cx(0, 1) 
    This entangles positions 0 and 1 in a Bell state
"""

from qiskit import QuantumCircuit, transpile
from qiskit_aer import Aer

#create TicTacToe blueprint:
class QuantumTicTacToe:

    def __init__(self):
        self.board = [' ' for _ in range(9)]
        self.entangled_pairs = {}

    #set up the board:
    def display(self):
        print("\n")
        for i in range(0, 9, 3):
            print(f" {self.board[i]} | {self.board[i + 1]} | {self.board[i + 2]}")
            if i < 6:
                print("---|---|---")

    #generic tile move:
    def quantum_move(self):
        qc1 = QuantumCircuit(1, 1)
        qc1.h(0)  # Superposition: equal chance of X or O
        qc1.measure(0, 0)
        outcome = simulation(qc1)
        return outcome

    #entangle positions a and b:
    def entangle(self, a, b):
        if self.board[a] == ' ' or self.board[b] == ' ':
            print("1 or both tiles are unfilled \n")
            return
        #make a dict with a and b --> {a, b}:
        self.entangled_pairs[0] = a
        self.entangled_pairs[1] = b

        self.collapse_entangled_pair(a, b)

    #create entangled Bell pair:
    def collapse_entangled_pair(self, a, b):
        qc2 = QuantumCircuit(2, 2)
        qc2.h(0)
        qc2.cx(0, 1) #Bell state
        qc2.measure_all() #collapse a and b
        outcome = simulation(qc2) #like '11' or '00'

        val_a = 'X' if outcome[1] == '1' else 'O' #if a collapses to 1  --> a = 'X' vice versa
        val_b = 'X' if outcome[0] == '1' else 'O'

        #change tiles a and b accordingly:
        self.board[a] = val_a 
        self.board[b] = val_b
        
        print(f"You entangled ", a, "and ", b, "and got ", val_a, val_b)

    #make move:
    def make_move(self, tile):
        if self.board[tile] != ' ':
            print("Tile already filled \n")
            return
        result = self.quantum_move()
        self.board[tile] = 'X' if result == '1' else 'O'
        return result

    def move_checker(self, array, result, tile):
        result = self.make_move(tile)
        for i in range(0, 8):
            if i == tile: #array[0] = 1
                if result == '1':
                    array[i] = 10
                else:
                    array[i] = 0
        game_won = win_detection(array)
        return game_won

    def is_full(self):
        return all(cell != ' ' for cell in self.board)

In [ ]:
#adding AI:

In [437]:
#play the game:
game = QuantumTicTacToe()
game_won = False
array = np.array([1,2,3,4,5,6,7,8,9])

print("Welcome to Quantum Tic-Tac-Toe")
print("You can:")
print("- Choose tiles between 0 and 8 which land O or X with a 50-50 chance each")
print("- Entangle placed tiles")

while not game.is_full():
    game.display()
    move = input("Input 0 - 8 to place a tile, or entangle tile_1 tile_2): ")
    if move.startswith("entangle"):
        _, a, b = move.split()
        game.entangle(int(a), int(b))
    int_move = int(move)
    if int_move < 9:
        game_won = game.move_checker(array, result, int_move)
        if game_won == True:
            print("Game over")
            break
    elif int_move >= 9:
        print("Out of range \n")
    else:
        print("Invalid input") #currently not working

if game.is_full():
    print("Final board: \n")
    game.display()

Welcome to Quantum Tic-Tac-Toe
You can:
- Choose tiles between 0 and 8 which land O or X with a 50-50 chance each
- Entangle placed tiles


   |   |  
---|---|---
   |   |  
---|---|---
   |   |  


 O |   |  
---|---|---
   |   |  
---|---|---
   |   |  


 O | O |  
---|---|---
   |   |  
---|---|---
   |   |  


 O | O | X
---|---|---
   |   |  
---|---|---
   |   |  
Tile already filled 

Game over
0 0 0
4 5 6
7 8 9
Game over
